In [190]:
import numpy as np

In [191]:
y = np.arange(35).reshape(5,7)
y

array([[ 0,  1,  2,  3,  4,  5,  6],
       [ 7,  8,  9, 10, 11, 12, 13],
       [14, 15, 16, 17, 18, 19, 20],
       [21, 22, 23, 24, 25, 26, 27],
       [28, 29, 30, 31, 32, 33, 34]])

In [192]:
y[[0, 2, 4], [0, 1, 2]]

array([ 0, 15, 30])

In [193]:
matrix = np.random.randint(1, 25, (4, 4))
window = (2, 2)
strides = (1, 1)
channels_out = 3
matrix

array([[ 3, 17, 20, 21],
       [ 1,  1, 23,  4],
       [12, 24,  7, 15],
       [ 1, 15, 23,  1]])

In [194]:
i = np.repeat(np.arange(window[0]), window[1])
col = np.tile(i, (channels_out))[:, None]
i = col + np.repeat(np.arange(3), 3)

j = np.tile(np.tile(np.arange(window[1]), window[0]), channels_out)
j = j[:, None] + np.tile(np.arange(3), 3)

# j[None, :].T + np.arange(3)



In [195]:
def get_indices(channels_out, shape, window, strides, padding):
    out_h = int((shape[0] + 2 * padding[0] - window[0]) / strides[0]) + 1
    out_w = int((shape[1] + 2 * padding[1] - window[1]) / strides[1]) + 1
    
    # i indices
    
    # base for each slide
    slide_i = np.repeat(np.arange(window[0]), window[1])
    # for each channel the same
    slide_channels_i = np.tile(slide_i, channels_out)
    # move horizontally
    i = slide_channels_i[:, None] + strides[0] * np.repeat(np.arange(out_h), out_w)
    
    # j indices
    
    # base for each slide
    slide_j = np.tile(np.arange(window[1]), window[0])
    # for each channel
    slide_channels_j = np.tile(slide_j, channels_out)
    # move vertically
    j = slide_channels_j[:, None] + strides[1] * np.tile(np.arange(out_w), out_h)

    # channel indices
    c = np.repeat(np.arange(channels_out), window[0] * window[1])[:, None]
    return i, j, c

get_indices(3, (4, 4), (2, 2), (1, 1), (0, 0))

(array([[0, 0, 0, 1, 1, 1, 2, 2, 2],
        [0, 0, 0, 1, 1, 1, 2, 2, 2],
        [1, 1, 1, 2, 2, 2, 3, 3, 3],
        [1, 1, 1, 2, 2, 2, 3, 3, 3],
        [0, 0, 0, 1, 1, 1, 2, 2, 2],
        [0, 0, 0, 1, 1, 1, 2, 2, 2],
        [1, 1, 1, 2, 2, 2, 3, 3, 3],
        [1, 1, 1, 2, 2, 2, 3, 3, 3],
        [0, 0, 0, 1, 1, 1, 2, 2, 2],
        [0, 0, 0, 1, 1, 1, 2, 2, 2],
        [1, 1, 1, 2, 2, 2, 3, 3, 3],
        [1, 1, 1, 2, 2, 2, 3, 3, 3]]),
 array([[0, 1, 2, 0, 1, 2, 0, 1, 2],
        [1, 2, 3, 1, 2, 3, 1, 2, 3],
        [0, 1, 2, 0, 1, 2, 0, 1, 2],
        [1, 2, 3, 1, 2, 3, 1, 2, 3],
        [0, 1, 2, 0, 1, 2, 0, 1, 2],
        [1, 2, 3, 1, 2, 3, 1, 2, 3],
        [0, 1, 2, 0, 1, 2, 0, 1, 2],
        [1, 2, 3, 1, 2, 3, 1, 2, 3],
        [0, 1, 2, 0, 1, 2, 0, 1, 2],
        [1, 2, 3, 1, 2, 3, 1, 2, 3],
        [0, 1, 2, 0, 1, 2, 0, 1, 2],
        [1, 2, 3, 1, 2, 3, 1, 2, 3]]),
 array([[0],
        [0],
        [0],
        [0],
        [1],
        [1],
        [1],
        [1],
    

In [196]:
def add_padding(x, padding):
    # accepts 3d array
    return np.pad(x, ((0, 0), (padding[1], padding[1]), (padding[0], padding[0])))

In [197]:
m = np.array([matrix, np.flip(matrix)])

In [198]:
matrix

array([[ 3, 17, 20, 21],
       [ 1,  1, 23,  4],
       [12, 24,  7, 15],
       [ 1, 15, 23,  1]])

In [199]:
def im2col(x, window, strides, padding):
    padded = add_padding(x, padding)
    i, j, c = get_indices(x.shape[0], x.shape[1:], window, strides, padding)
    cols = padded[c, i, j]
    return cols

im2col(m, (2, 2), (1, 1), (0, 0))

array([[ 3, 17, 20,  1,  1, 23, 12, 24,  7],
       [17, 20, 21,  1, 23,  4, 24,  7, 15],
       [ 1,  1, 23, 12, 24,  7,  1, 15, 23],
       [ 1, 23,  4, 24,  7, 15, 15, 23,  1],
       [ 1, 23, 15, 15,  7, 24,  4, 23,  1],
       [23, 15,  1,  7, 24, 12, 23,  1,  1],
       [15,  7, 24,  4, 23,  1, 21, 20, 17],
       [ 7, 24, 12, 23,  1,  1, 20, 17,  3]])

In [200]:
m

array([[[ 3, 17, 20, 21],
        [ 1,  1, 23,  4],
        [12, 24,  7, 15],
        [ 1, 15, 23,  1]],

       [[ 1, 23, 15,  1],
        [15,  7, 24, 12],
        [ 4, 23,  1,  1],
        [21, 20, 17,  3]]])

In [282]:
def max_pooling(x, window, strides, padding):
    
    n_H = int((x.shape[1] + 2 * padding[0] - window[0])/ strides[0]) + 1
    n_W = int((x.shape[2] + 2 * padding[1] - window[1])/ strides[1]) + 1
    
    x_col = im2col(x, window, strides, padding)
    x_col = x_col.reshape(x.shape[0], window[0] * window[1], -1)
    print(x_col)
    a_pol = np.max(x_col, axis=1)
    return a_pol.reshape((x.shape[0], n_H, n_W)), x_col.argmax(axis=1).reshape((x.shape[0], n_H, n_W))
pooled, indices = max_pooling(m, (2, 2), (1, 1), (0, 0))
pooled, indices

[[[ 3 17 20  1  1 23 12 24  7]
  [17 20 21  1 23  4 24  7 15]
  [ 1  1 23 12 24  7  1 15 23]
  [ 1 23  4 24  7 15 15 23  1]]

 [[ 1 23 15 15  7 24  4 23  1]
  [23 15  1  7 24 12 23  1  1]
  [15  7 24  4 23  1 21 20 17]
  [ 7 24 12 23  1  1 20 17  3]]]


(array([[[17, 23, 23],
         [24, 24, 23],
         [24, 24, 23]],
 
        [[23, 24, 24],
         [23, 24, 24],
         [23, 23, 17]]]),
 array([[[1, 3, 2],
         [3, 2, 0],
         [1, 0, 2]],
 
        [[1, 3, 2],
         [3, 1, 0],
         [1, 0, 2]]]))

In [278]:
indices.flatten()

array([1, 3, 2, 3, 2, 0, 1, 0, 2, 1, 3, 2, 3, 1, 0, 1, 0, 2])

In [ ]:
def pool_backward(x, indices, error, window, strides, padding):
    n_H = int((x.shape[1] + 2 * padding[0] - window[0])/ strides[0]) + 1
    n_W = int((x.shape[2] + 2 * padding[1] - window[1])/ strides[1]) + 1
    
    dx = np.zeros(x.shape)
    
pool_backward(np.ones_like(pooled), indices, np.ones_like(pooled), (2, 2), (1, 1), (0, 0))

In [261]:
def col2im(x_col, x_shape, window, strides, padding):
    H_padded = x_shape[1] + 2 * padding[0]
    W_padded = x_shape[2] + 2 * padding[1]
    
    new_x = np.zeros((x_shape[0], H_padded, W_padded))
    print(new_x.shape)
    
    i, j, d = get_indices(x_shape[0], x_shape[1:], window, strides, padding)
    np.add.at(new_x, (d, i, j), x_col)
    if padding[0] != 0 or padding[1] != 0:
        return new_x[:, padding[0]:-padding[0], padding[1]:-padding[1]]
    return new_x

In [262]:
x_col = im2col(m, (2, 2), (1, 1), (0, 0))
x_col

array([[ 3, 17, 20,  1,  1, 23, 12, 24,  7],
       [17, 20, 21,  1, 23,  4, 24,  7, 15],
       [ 1,  1, 23, 12, 24,  7,  1, 15, 23],
       [ 1, 23,  4, 24,  7, 15, 15, 23,  1],
       [ 1, 23, 15, 15,  7, 24,  4, 23,  1],
       [23, 15,  1,  7, 24, 12, 23,  1,  1],
       [15,  7, 24,  4, 23,  1, 21, 20, 17],
       [ 7, 24, 12, 23,  1,  1, 20, 17,  3]])

In [263]:
col2im(x_col, m.shape, (2, 2), (1, 1), (0, 0))

(2, 4, 4)


array([[[ 3., 34., 40., 21.],
        [ 2.,  4., 92.,  8.],
        [24., 96., 28., 30.],
        [ 1., 30., 46.,  1.]],

       [[ 1., 46., 30.,  1.],
        [30., 28., 96., 24.],
        [ 8., 92.,  4.,  2.],
        [21., 40., 34.,  3.]]])